# Transportation Optimization and Urban Network Analysis

**Tier 0 - Free Tier (Google Colab / Amazon SageMaker Studio Lab)**

## Overview

This notebook introduces transportation network analysis and optimization methods for urban planning. You'll work with street networks, optimize routes, simulate traffic, and analyze accessibility patterns.

**What you'll learn:**
- Street network representation and analysis
- Shortest path algorithms (Dijkstra, A*)
- Vehicle Routing Problem (VRP) for delivery optimization
- Traffic flow simulation and congestion modeling
- Travel time isochrones and accessibility analysis
- Public transit route optimization
- Network centrality and critical infrastructure identification
- Multi-objective route optimization (time, distance, cost)

**Runtime:** 25-35 minutes

**Requirements:** `networkx`, `matplotlib`, `numpy`, `pandas`, `scipy`

**Note:** This notebook uses synthetic networks for speed. Tier 1+ includes OSMnx for real OpenStreetMap data.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Set random seed
np.random.seed(42)

print("Environment ready for transportation network analysis")

## 1. Generate Synthetic Urban Street Network

Create a realistic urban grid network with nodes (intersections) and edges (streets) with attributes like distance and speed limits.

In [ ]:
def generate_urban_network(grid_size=10, avg_block_length=100, speed_limit_mean=40):
    """
    Generate synthetic urban street network
    
    Args:
        grid_size: number of blocks per dimension
        avg_block_length: average block length in meters
        speed_limit_mean: mean speed limit in km/h
    """
    G = nx.grid_2d_graph(grid_size, grid_size)
    
    # Convert to directed graph for one-way streets
    G = G.to_directed()
    
    # Add spatial coordinates
    pos = {}
    for node in G.nodes():
        x, y = node
        # Add some jitter to make it more realistic
        pos[node] = (
            x * avg_block_length + np.random.normal(0, avg_block_length * 0.05),
            y * avg_block_length + np.random.normal(0, avg_block_length * 0.05)
        )
    
    nx.set_node_attributes(G, pos, 'pos')
    
    # Add edge attributes
    for u, v in G.edges():
        # Distance (Euclidean)
        u_pos = pos[u]
        v_pos = pos[v]
        distance = np.sqrt((u_pos[0] - v_pos[0])**2 + (u_pos[1] - v_pos[1])**2)
        
        # Speed limit (with variation for arterials vs local streets)
        is_arterial = (u[0] % 3 == 0) or (u[1] % 3 == 0)  # Every 3rd street is arterial
        speed_limit = speed_limit_mean * (1.3 if is_arterial else 1.0)
        speed_limit += np.random.normal(0, 5)
        speed_limit = max(20, speed_limit)  # Minimum 20 km/h
        
        # Travel time (in minutes)
        travel_time = (distance / 1000) / (speed_limit / 60)
        
        G[u][v]['distance'] = distance
        G[u][v]['speed_limit'] = speed_limit
        G[u][v]['travel_time'] = travel_time
        G[u][v]['capacity'] = 1000 if is_arterial else 500  # vehicles/hour
        G[u][v]['flow'] = 0  # Initial traffic flow
    
    return G

# Generate network
G_city = generate_urban_network(grid_size=15, avg_block_length=100, speed_limit_mean=40)

print(f"Urban Network Statistics:")
print(f"  Nodes (intersections): {G_city.number_of_nodes()}")
print(f"  Edges (street segments): {G_city.number_of_edges()}")
print(f"  Total network length: {sum(nx.get_edge_attributes(G_city, 'distance').values()) / 1000:.2f} km")
print(f"  Average block length: {np.mean(list(nx.get_edge_attributes(G_city, 'distance').values())):.1f} m")

In [ ]:
# Visualize network
fig, ax = plt.subplots(figsize=(12, 12))

pos = nx.get_node_attributes(G_city, 'pos')

# Draw edges with different colors for arterials
arterial_edges = [(u, v) for u, v in G_city.edges() if G_city[u][v]['capacity'] > 700]
local_edges = [(u, v) for u, v in G_city.edges() if G_city[u][v]['capacity'] <= 700]

nx.draw_networkx_edges(G_city, pos, edgelist=local_edges, width=1, alpha=0.5, 
                       edge_color='gray', arrows=False, ax=ax)
nx.draw_networkx_edges(G_city, pos, edgelist=arterial_edges, width=2.5, alpha=0.7, 
                       edge_color='orange', arrows=False, ax=ax, label='Arterial roads')
nx.draw_networkx_nodes(G_city, pos, node_size=30, node_color='black', alpha=0.6, ax=ax)

ax.set_title('Synthetic Urban Street Network', fontsize=16)
ax.set_xlabel('Distance (meters)', fontsize=12)
ax.set_ylabel('Distance (meters)', fontsize=12)
ax.legend()
ax.axis('equal')

plt.tight_layout()
plt.show()

print("Orange lines = arterial roads (higher capacity), Gray lines = local streets")

## 2. Shortest Path Routing

Find optimal routes between origin and destination using Dijkstra's algorithm. Compare distance-minimizing vs time-minimizing routes.

In [ ]:
# Select random origin and destination
nodes = list(G_city.nodes())
origin = nodes[0]  # Bottom-left corner
destination = nodes[-1]  # Top-right corner

print(f"Route Planning:")
print(f"  Origin: {origin}")
print(f"  Destination: {destination}")

# Shortest path by distance
path_distance = nx.shortest_path(G_city, origin, destination, weight='distance')
distance_total = sum(G_city[path_distance[i]][path_distance[i+1]]['distance'] 
                     for i in range(len(path_distance)-1))
time_via_distance = sum(G_city[path_distance[i]][path_distance[i+1]]['travel_time'] 
                        for i in range(len(path_distance)-1))

# Shortest path by time
path_time = nx.shortest_path(G_city, origin, destination, weight='travel_time')
distance_via_time = sum(G_city[path_time[i]][path_time[i+1]]['distance'] 
                        for i in range(len(path_time)-1))
time_total = sum(G_city[path_time[i]][path_time[i+1]]['travel_time'] 
                 for i in range(len(path_time)-1))

print(f"\nDistance-Optimized Route:")
print(f"  Total distance: {distance_total:.0f} m ({distance_total/1000:.2f} km)")
print(f"  Travel time: {time_via_distance:.2f} min")
print(f"  Number of turns: {len(path_distance) - 1}")

print(f"\nTime-Optimized Route:")
print(f"  Total distance: {distance_via_time:.0f} m ({distance_via_time/1000:.2f} km)")
print(f"  Travel time: {time_total:.2f} min")
print(f"  Number of turns: {len(path_time) - 1}")

print(f"\nTime savings: {time_via_distance - time_total:.2f} min ({(time_via_distance - time_total)/time_via_distance*100:.1f}%)")
print(f"Extra distance: {distance_via_time - distance_total:.0f} m ({(distance_via_time - distance_total)/distance_total*100:.1f}%)")

In [ ]:
# Visualize both routes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

for ax, path, title in [(ax1, path_distance, 'Distance-Optimized Route'),
                         (ax2, path_time, 'Time-Optimized Route')]:
    # Draw network
    nx.draw_networkx_edges(G_city, pos, width=0.5, alpha=0.3, edge_color='gray', arrows=False, ax=ax)
    nx.draw_networkx_nodes(G_city, pos, node_size=10, node_color='lightgray', ax=ax)
    
    # Draw path
    path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
    nx.draw_networkx_edges(G_city, pos, edgelist=path_edges, width=3, 
                           edge_color='red', arrows=True, arrowsize=15, ax=ax)
    
    # Highlight origin and destination
    nx.draw_networkx_nodes(G_city, pos, nodelist=[origin], node_size=200, 
                           node_color='green', label='Origin', ax=ax)
    nx.draw_networkx_nodes(G_city, pos, nodelist=[destination], node_size=200, 
                           node_color='blue', label='Destination', ax=ax)
    
    ax.set_title(title, fontsize=14)
    ax.legend(loc='upper left')
    ax.axis('equal')
    ax.axis('off')

plt.tight_layout()
plt.show()

## 3. Vehicle Routing Problem (VRP)

Optimize delivery routes for multiple stops. This is the classic "traveling salesman problem" variant used in logistics.

In [ ]:
def solve_vrp_greedy(G, depot, delivery_locations):
    """
    Solve Vehicle Routing Problem using greedy nearest neighbor heuristic
    """
    unvisited = set(delivery_locations)
    current = depot
    route = [depot]
    total_distance = 0
    total_time = 0
    
    while unvisited:
        # Find nearest unvisited location
        nearest = None
        min_distance = float('inf')
        
        for location in unvisited:
            try:
                distance = nx.shortest_path_length(G, current, location, weight='distance')
                if distance < min_distance:
                    min_distance = distance
                    nearest = location
            except nx.NetworkXNoPath:
                continue
        
        if nearest is None:
            break
        
        # Move to nearest location
        path = nx.shortest_path(G, current, nearest, weight='distance')
        segment_distance = sum(G[path[i]][path[i+1]]['distance'] for i in range(len(path)-1))
        segment_time = sum(G[path[i]][path[i+1]]['travel_time'] for i in range(len(path)-1))
        
        route.append(nearest)
        total_distance += segment_distance
        total_time += segment_time
        unvisited.remove(nearest)
        current = nearest
    
    # Return to depot
    if current != depot:
        path = nx.shortest_path(G, current, depot, weight='distance')
        segment_distance = sum(G[path[i]][path[i+1]]['distance'] for i in range(len(path)-1))
        segment_time = sum(G[path[i]][path[i+1]]['travel_time'] for i in range(len(path)-1))
        route.append(depot)
        total_distance += segment_distance
        total_time += segment_time
    
    return route, total_distance, total_time

# Define delivery scenario
depot = (7, 7)  # Central location
n_deliveries = 10
delivery_locations = [nodes[i] for i in np.random.choice(len(nodes), n_deliveries, replace=False)]

print(f"Vehicle Routing Problem:")
print(f"  Depot: {depot}")
print(f"  Number of delivery stops: {n_deliveries}")

# Solve VRP
route, total_distance, total_time = solve_vrp_greedy(G_city, depot, delivery_locations)

print(f"\nOptimized Route (Greedy Nearest Neighbor):")
print(f"  Total distance: {total_distance:.0f} m ({total_distance/1000:.2f} km)")
print(f"  Total travel time: {total_time:.2f} min ({total_time/60:.2f} hours)")
print(f"  Stops: {len(route)}")
print(f"  Route: {' → '.join([str(node) for node in route[:5]])} ... {' → '.join([str(node) for node in route[-3:]])}")

In [ ]:
# Visualize VRP solution
fig, ax = plt.subplots(figsize=(12, 12))

# Draw network
nx.draw_networkx_edges(G_city, pos, width=0.5, alpha=0.2, edge_color='gray', arrows=False, ax=ax)
nx.draw_networkx_nodes(G_city, pos, node_size=10, node_color='lightgray', ax=ax)

# Draw delivery route
for i in range(len(route) - 1):
    start, end = route[i], route[i+1]
    path = nx.shortest_path(G_city, start, end, weight='distance')
    path_edges = [(path[j], path[j+1]) for j in range(len(path)-1)]
    
    # Color gradient for route order
    color = plt.cm.viridis(i / len(route))
    nx.draw_networkx_edges(G_city, pos, edgelist=path_edges, width=2, 
                           edge_color=[color], arrows=True, arrowsize=10, ax=ax)

# Highlight stops
nx.draw_networkx_nodes(G_city, pos, nodelist=[depot], node_size=300, 
                       node_color='red', label='Depot', ax=ax, edgecolors='black', linewidths=2)
nx.draw_networkx_nodes(G_city, pos, nodelist=delivery_locations, node_size=150, 
                       node_color='yellow', label='Delivery stops', ax=ax, edgecolors='black', linewidths=1.5)

# Add route order labels
for i, node in enumerate(route[1:-1], 1):
    if node in delivery_locations:
        x, y = pos[node]
        ax.text(x, y, str(i), fontsize=10, ha='center', va='center', fontweight='bold')

ax.set_title(f'Vehicle Routing Solution ({n_deliveries} stops, {total_distance/1000:.2f} km)', fontsize=16)
ax.legend(loc='upper left', fontsize=12)
ax.axis('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

print("Route visualization shows delivery sequence (numbers) and optimal path (colored arrows)")

## 4. Traffic Flow Simulation

Simulate traffic demand and analyze congestion patterns. Calculate Volume-to-Capacity (V/C) ratios for each street segment.

In [ ]:
def simulate_traffic_demand(G, n_trips=1000):
    """
    Simulate traffic demand by generating random OD pairs and routing
    """
    nodes = list(G.nodes())
    
    # Reset flows
    for u, v in G.edges():
        G[u][v]['flow'] = 0
    
    # Generate trips
    for _ in range(n_trips):
        origin, destination = np.random.choice(nodes, 2, replace=False)
        
        try:
            # Find shortest path by time
            path = nx.shortest_path(G, origin, destination, weight='travel_time')
            
            # Add flow to each edge
            for i in range(len(path) - 1):
                G[path[i]][path[i+1]]['flow'] += 1
        except nx.NetworkXNoPath:
            continue
    
    # Calculate V/C ratios
    for u, v in G.edges():
        flow = G[u][v]['flow']
        capacity = G[u][v]['capacity']
        G[u][v]['vc_ratio'] = flow / capacity
    
    return G

# Simulate traffic
G_city = simulate_traffic_demand(G_city, n_trips=2000)

# Analyze congestion
flows = [data['flow'] for _, _, data in G_city.edges(data=True)]
vc_ratios = [data['vc_ratio'] for _, _, data in G_city.edges(data=True)]

print(f"Traffic Simulation Results (2000 trips):")
print(f"  Average flow per edge: {np.mean(flows):.1f} vehicles/hour")
print(f"  Max flow on any edge: {np.max(flows):.0f} vehicles/hour")
print(f"  Average V/C ratio: {np.mean(vc_ratios):.3f}")
print(f"  Max V/C ratio: {np.max(vc_ratios):.3f}")
print(f"  Congested links (V/C > 0.8): {sum(1 for vc in vc_ratios if vc > 0.8)} ({sum(1 for vc in vc_ratios if vc > 0.8)/len(vc_ratios)*100:.1f}%)")

In [ ]:
# Visualize traffic congestion
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Plot 1: V/C ratio heatmap
vc_ratios = {(u, v): data['vc_ratio'] for u, v, data in G_city.edges(data=True)}
edge_colors = [vc_ratios[edge] for edge in G_city.edges()]

nx.draw_networkx_nodes(G_city, pos, node_size=10, node_color='black', ax=ax1)
edges = nx.draw_networkx_edges(
    G_city, pos, 
    edge_color=edge_colors,
    edge_cmap=plt.cm.RdYlGn_r,
    edge_vmin=0,
    edge_vmax=1.0,
    width=2,
    arrows=False,
    ax=ax1
)

sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlGn_r, norm=plt.Normalize(vmin=0, vmax=1.0))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax1)
cbar.set_label('V/C Ratio (Volume/Capacity)', fontsize=11)

ax1.set_title('Traffic Congestion Map (V/C Ratio)', fontsize=14)
ax1.axis('equal')
ax1.axis('off')

# Plot 2: V/C distribution
ax2.hist(vc_ratios, bins=30, edgecolor='black', alpha=0.7)
ax2.axvline(0.8, color='red', linestyle='--', linewidth=2, label='Congestion threshold (0.8)')
ax2.set_xlabel('V/C Ratio', fontsize=12)
ax2.set_ylabel('Number of street segments', fontsize=12)
ax2.set_title('Distribution of V/C Ratios', fontsize=14)
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRed segments = congested (V/C > 0.8), Yellow = moderate, Green = free flow")

## 5. Accessibility Analysis: Travel Time Isochrones

Calculate how far you can travel from a given point in a fixed time (e.g., 15-minute walk).

In [ ]:
def calculate_isochrone(G, origin, max_time, weight='travel_time'):
    """
    Calculate isochrone: all nodes reachable within max_time
    """
    # Compute shortest paths
    lengths = nx.single_source_dijkstra_path_length(G, origin, weight=weight)
    
    # Filter by time threshold
    reachable_nodes = {node: time for node, time in lengths.items() if time <= max_time}
    
    return reachable_nodes

# Calculate isochrones from city center
center = (7, 7)
isochrone_5min = calculate_isochrone(G_city, center, max_time=5)
isochrone_10min = calculate_isochrone(G_city, center, max_time=10)
isochrone_15min = calculate_isochrone(G_city, center, max_time=15)

print(f"Accessibility Analysis from {center}:")
print(f"  Locations within 5 minutes: {len(isochrone_5min)}")
print(f"  Locations within 10 minutes: {len(isochrone_10min)}")
print(f"  Locations within 15 minutes: {len(isochrone_15min)}")
print(f"  Total coverage (15 min): {len(isochrone_15min) / G_city.number_of_nodes() * 100:.1f}% of city")

In [ ]:
# Visualize isochrones
fig, ax = plt.subplots(figsize=(12, 12))

# Draw network
nx.draw_networkx_edges(G_city, pos, width=0.5, alpha=0.2, edge_color='gray', arrows=False, ax=ax)

# Draw isochrone zones
nodes_5min = list(isochrone_5min.keys())
nodes_10min = [n for n in isochrone_10min.keys() if n not in isochrone_5min]
nodes_15min = [n for n in isochrone_15min.keys() if n not in isochrone_10min]
nodes_outside = [n for n in G_city.nodes() if n not in isochrone_15min]

nx.draw_networkx_nodes(G_city, pos, nodelist=nodes_outside, node_size=80, 
                       node_color='lightgray', alpha=0.3, ax=ax)
nx.draw_networkx_nodes(G_city, pos, nodelist=nodes_15min, node_size=100, 
                       node_color='yellow', alpha=0.6, label='10-15 min', ax=ax)
nx.draw_networkx_nodes(G_city, pos, nodelist=nodes_10min, node_size=120, 
                       node_color='orange', alpha=0.7, label='5-10 min', ax=ax)
nx.draw_networkx_nodes(G_city, pos, nodelist=nodes_5min, node_size=140, 
                       node_color='red', alpha=0.8, label='0-5 min', ax=ax)
nx.draw_networkx_nodes(G_city, pos, nodelist=[center], node_size=300, 
                       node_color='darkblue', label='Origin', ax=ax, edgecolors='white', linewidths=2)

ax.set_title('Travel Time Isochrones from City Center', fontsize=16)
ax.legend(loc='upper left', fontsize=12)
ax.axis('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

print("Isochrones show areas reachable within 5, 10, and 15 minutes by car")

## 6. Network Centrality: Critical Infrastructure

Identify critical intersections and roads using centrality metrics.

In [ ]:
# Calculate centrality metrics
degree_centrality = nx.degree_centrality(G_city)
betweenness_centrality = nx.betweenness_centrality(G_city, weight='travel_time')
closeness_centrality = nx.closeness_centrality(G_city, distance='travel_time')

# Top critical nodes
top_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
top_closeness = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

print("Critical Infrastructure Analysis:")
print("\nTop 10 Critical Intersections (by betweenness centrality):")
print("These nodes are most important for network connectivity")
for i, (node, centrality) in enumerate(top_betweenness, 1):
    print(f"  {i}. {node}: {centrality:.4f}")

print("\nTop 10 Most Accessible Locations (by closeness centrality):")
print("These nodes are easiest to reach from anywhere in the city")
for i, (node, centrality) in enumerate(top_closeness, 1):
    print(f"  {i}. {node}: {centrality:.4f}")

In [ ]:
# Visualize centrality
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Betweenness centrality
node_sizes_1 = [betweenness_centrality[node] * 5000 + 20 for node in G_city.nodes()]
node_colors_1 = [betweenness_centrality[node] for node in G_city.nodes()]

nx.draw_networkx_edges(G_city, pos, width=0.5, alpha=0.2, edge_color='gray', arrows=False, ax=ax1)
nx.draw_networkx_nodes(G_city, pos, node_size=node_sizes_1, node_color=node_colors_1,
                       cmap='Reds', alpha=0.8, ax=ax1)

ax1.set_title('Betweenness Centrality (Critical Nodes)', fontsize=14)
ax1.axis('equal')
ax1.axis('off')

# Closeness centrality
node_sizes_2 = [closeness_centrality[node] * 500 + 20 for node in G_city.nodes()]
node_colors_2 = [closeness_centrality[node] for node in G_city.nodes()]

nx.draw_networkx_edges(G_city, pos, width=0.5, alpha=0.2, edge_color='gray', arrows=False, ax=ax2)
nx.draw_networkx_nodes(G_city, pos, node_size=node_sizes_2, node_color=node_colors_2,
                       cmap='Blues', alpha=0.8, ax=ax2)

ax2.set_title('Closeness Centrality (Accessibility)', fontsize=14)
ax2.axis('equal')
ax2.axis('off')

plt.tight_layout()
plt.show()

print("Node size and color indicate centrality (larger/darker = more critical/accessible)")

## Summary and Next Steps

### What We've Accomplished

1. **Street Network Modeling**
   - Generated synthetic urban grid network (15×15 blocks)
   - Modeled arterial and local streets with realistic attributes
   - Calculated distances, travel times, and capacities

2. **Routing Optimization**
   - Computed shortest paths by distance and time
   - Compared trade-offs between route length and travel time
   - Demonstrated multi-objective optimization

3. **Vehicle Routing Problem (VRP)**
   - Solved 10-stop delivery optimization using greedy heuristic
   - Minimized total travel distance and time
   - Visualized optimal route sequence

4. **Traffic Simulation**
   - Simulated 2,000 origin-destination trips
   - Calculated traffic flow and V/C ratios
   - Identified congested corridors

5. **Accessibility Analysis**
   - Computed travel time isochrones (5, 10, 15 minutes)
   - Measured spatial coverage from city center
   - Analyzed reachability patterns

6. **Network Criticality**
   - Identified critical intersections using betweenness centrality
   - Found most accessible locations using closeness centrality
   - Prioritized infrastructure for maintenance/improvement

### Key Insights

- **Arterial roads are critical**: Higher capacity streets carry disproportionate traffic
- **Time-optimal ≠ distance-optimal**: Faster routes often take longer paths
- **Centrality identifies bottlenecks**: High betweenness nodes are vulnerable to disruption
- **Accessibility varies spatially**: Central locations have shorter average travel times
- **Greedy heuristics are fast but suboptimal**: Real VRP needs metaheuristics or exact methods

### Limitations

- Synthetic network doesn't capture real urban complexity
- No signal timing, turn restrictions, or elevation
- Static analysis (no time-varying congestion)
- Deterministic travel times (no variability)
- Single mode (no transit, walking, cycling)

### Progression Path

**Tier 1** - Real-world transportation networks
- OSMnx for OpenStreetMap data (any city worldwide)
- Multi-modal routing (car, transit, bike, walk)
- Dynamic traffic assignment with time-varying demand
- Public transit schedule integration (GTFS data)

**Tier 2** - AWS-integrated transportation platform
- S3 for OSM data storage
- Lambda for on-demand route calculation
- Real-time traffic data from HERE/TomTom APIs
- Interactive map visualizations with Folium/Kepler.gl

**Tier 3** - Production-scale mobility analytics
- CloudFormation stack (EC2, RDS, ElastiCache, API Gateway)
- Large-scale VRP with genetic algorithms or simulated annealing
- Real-time traffic prediction with machine learning
- Equity analysis (accessibility by income, race, age)
- Scenario planning for new infrastructure projects
- Integration with ride-hailing/delivery APIs (Uber, DoorDash)

### Additional Resources

- OSMnx documentation: https://osmnx.readthedocs.io/
- NetworkX documentation: https://networkx.org/
- "Transport Planning with Graph Neural Networks" (recent research)
- "The Vehicle Routing Problem" by Toth and Vigo
- OpenStreetMap: https://www.openstreetmap.org/
- GTFS (transit data): https://gtfs.org/